# Домашнее задание к занятию "Классификация: Логистическая регрессия и SVM"

In [4]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

#### Загружаем данные

In [5]:
df = pd.read_csv('adult.csv', na_values='?') # Загрузил датасет и подставил вместо ? - отстутсвие значения
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


#### Выбираем колонки для работы
Не стал брать capital-gain и capital-loss - там почти все значения нули, не думают что положительно скажутся на обучении модели.
fnlwgt - тоже не стал брать, не вижу корелляции с income
hours-per-week - тоже не брал, мне кажется уровень дохода от этого не зависит

In [6]:
selectedCol = df[['age', 'workclass', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']]
selectedCol

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,native-country,income
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,>50K
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,United-States,<=50K


#### Проверяем на отсутствие значений

In [7]:
selectedCol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   education        48842 non-null  object
 3   educational-num  48842 non-null  int64 
 4   marital-status   48842 non-null  object
 5   occupation       46033 non-null  object
 6   relationship     48842 non-null  object
 7   race             48842 non-null  object
 8   gender           48842 non-null  object
 9   native-country   47985 non-null  object
 10  income           48842 non-null  object
dtypes: int64(2), object(9)
memory usage: 4.1+ MB


In [8]:
selectedCol = selectedCol.dropna()
selectedCol

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,native-country,income
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,>50K
5,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,United-States,<=50K


#### Убираем строки с пустыми значениям
Строк с пустыми значениями всего 7% - я думаю их можно просто удалить

#### Делаем X

In [9]:
X = pd.get_dummies( selectedCol, columns = [ 'workclass',  'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country'] )
X

,age,educational-num,gender,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,7,Male,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,9,Male,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,12,Male,>50K,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,10,Male,>50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,6,Male,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,12,Female,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48838,40,9,Male,>50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48839,58,9,Female,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48840,22,9,Male,<=50K,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit( X['gender'] ) 

X['gender'] = le.transform(X['gender']) # Через LabelEncoder преобразовываем пол в число

del X['income'] # Это целевое значение - удаляем из X
X

,age,educational-num,gender,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,7,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,9,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,12,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,10,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,6,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,12,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48838,40,9,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48839,58,9,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48840,22,9,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
le.fit( selectedCol['income'] )

y = pd.Series( data = le.transform( selectedCol['income'] ) ) # Делаем y из income и преобразуем в числовые значения
y

0        0
1        0
2        1
3        1
4        0
        ..
45217    0
45218    1
45219    0
45220    0
45221    1
Length: 45222, dtype: int64

#### Данные для обучения готовы 

In [12]:
# Подключаем необходимые библиотеке с моделями

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) # Делим выборки

In [14]:
# Создаем экземпляры моделей
model_LR = LogisticRegression()
model_SVC = SVC()

In [15]:
# Обучаем модели
model_LR.fit(X_train, y_train)
model_SVC.fit(X_train, y_train)

SVC()

#### Оценка модели логистической регрессии

In [16]:
model_LR.score(X_train, y_train)

0.8271022526241302

#### Оценка модели SVC

In [17]:
model_SVC.score(X_train, y_train)

0.8246550300743012

### Выводы:
Качество обученных моделей примерно одинаковы - 82%. Что я думаю весьма высоко.

Дополнительно оценим точность, с которой предсказывает модель. В сколько процентах случаях модель ошибается

In [18]:
from sklearn.metrics import accuracy_score # Загружаем метрику точности

accuracy_score(y_test, model_LR.predict(X_test)) # Вычисляем на сравнении тестовой выборки и предсказанной моделью LR

0.8316822925880064

In [19]:
accuracy_score(y_test, model_SVC.predict(X_test)) # Тоже самое для SVC

0.833185918981072

### Выводы:
Точность примерно одинаковая - 83%. Я думаю результат весьма хороший.